In [1]:
from MBDOE import *
from generalize_functions import CEOSModels


[    0.00] Initializing mpi-sppy


## Choose experiments and model

In [2]:
model_option = "PR"
if_poly = False

if model_option=="PR":
    if not if_poly:
        from hfc32_emimtf2n_PR import configuration
    else:
        from hfc32_emimtf2n_PR_polynomial import configuration
        
elif model_option=="SRK":
    if not if_poly:
        from hfc32_emimtf2n_SRK import configuration
    else:
        from hfc32_emimtf2n_SRK_polynomial import configuration
        

In [3]:
# Experiments 
data_file = './emimtf2n/R32/Final_Results/MBDoE/r32_emimtf2n_subset.csv'

path = './emimtf2n/R32/Final_Results/MBDoE/Params/'

# Decide on model 
#para_file = path + 'PR_params_1param_Opt1.csv'
#PR_type_opt = "1Param_Opt1"

#para_file = path + 'PR_params_1param_Opt2.csv'
#PR_type_opt = "1Param_Opt2"

#para_file = path + 'PR_params_1param_Opt2_GF.csv'
#PR_type_opt = "1Param_Opt2"

#para_file = path + 'PR_params_3params_Opt1.csv'
#PR_type_opt = "3Params_Opt1"

#para_file = path + 'PR_params_3params_Opt2.csv'
#PR_type_opt = "3Params_Opt2"

#para_file = path + 'PR_params_linTdep.csv'
#PR_type_opt = "Linear"

#para_file = path + 'PR_params_noTdep.csv'
#PR_type_opt = "No"

#para_file = path + 'PR_params_polyTdep.csv'
#PR_type_opt = "Polynomial"

# important model
para_file = path + 'PR_params_quadTdep.csv'
PR_type_opt = "Quadratic"

#para_file = path + 'SRK_params_1param_Opt1.csv'
#PR_type_opt = "1Param_Opt1"

#para_file = path + 'SRK_params_1param_Opt2.csv'
#PR_type_opt = "1Param_Opt2"

#para_file = path + 'SRK_params_3params_Opt1.csv'
#PR_type_opt = "3Params_Opt1"

#para_file = path + 'SRK_params_3params_Opt2.csv'
#PR_type_opt = "3Params_Opt2"

#para_file = path + 'SRK_params_linTdep.csv'
#PR_type_opt = "Linear"

#para_file = path + 'SRK_params_noTdep.csv'
#PR_type_opt = "No"


#para_file = path + 'SRK_params_polyTdep.csv'
#PR_type_opt = "Polynomial"

#para_file = path + 'SRK_params_quadTdep.csv'
#PR_type_opt = "Quadratic"



model_creation = CEOSModels(para_file, configuration, 
                         comp_1= "R32", comp_2 = "emimTf2N", 
                         x_comp_1="x_R32", x_comp_2="x_emimTf2N",  Model_type = model_option, Tdep_type=PR_type_opt)

## Run pyomo.doe

In [ ]:
data_exp = pd.read_csv(data_file)
print(len(data_exp))
print(data_exp.iloc[1])

In [ ]:
print(model_creation.param_name_dict)

In [ ]:
mbdoe_obj = MBDOE(data_file, model_creation)

exp_set = range(len(data_exp))
#exp_set = range(12)
#exp_set = [0,1,2,3]

#exp_set = 0
# init_T: 283-348; 323 is good; init_pressure: low mole fraction use 150000; high 399800; x_option: 0.4-0.6;0.3-0.55;
# SRK: init_temp_option=323, init_pressure_option= 399800, init_x_c1_option = 0.5

total_fim = mbdoe_obj.sumDOE(exp_set, scale_opt=True, record_name="SRK_polyTdep", 
                             init_temp_option=343, 
                             init_pressure_option= 150000,
                             init_x_c1_option = 0.4,
                             poly_option=True)
#total_fim = mbdoe_obj.doe(exp_set, scale=True)

In [ ]:
print(total_fim)
print(np.linalg.det(total_fim))
print(np.trace(total_fim))
print(np.linalg.eigvals(total_fim))

## Heatmap

In [14]:
# read prior

f = open('emimtf2n_FIM_info/PR_quadTdep.json')

data = json.load(f)

prior_info = data[0]['Total']

print(prior_info)
print(np.log10(np.linalg.det(prior_info)))

[[45575355309926.85, -10796186966711.326, 83445290175042.88, -7989512603441.533, 30035490945132.06, -5294333373880.846], [-10796186966711.326, 3888813365343.9434, -19011816632190.887, 2790548602457.38, -6577442271879.425, 1789815131395.5442], [83445290175042.88, -19011816632190.887, 153592012364861.78, -14134663295880.395, 55573688924934.195, -9411789134390.35], [-7989512603441.533, 2790548602457.38, -14134663295880.395, 2008060912146.0166, -4913769119004.827, 1291821973754.2095], [30035490945132.06, -6577442271879.425, 55573688924934.195, -4913769119004.827, 20211335904074.926, -3288302607352.0684], [-5294333373880.846, 1789815131395.5442, -9411789134390.35, 1291821973754.2095, -3288302607352.0684, 833744206456.5271]]
61.06068485550636


In [5]:
#data_exp = pd.read_csv(data_file)
#print(data_exp)

In [15]:
# model list 
#T_range = [273, 360]
#x_range = [0.1, 0.9]

T_range = [273, 283, 293, 303, 313, 323, 333, 343, 353, 360]
x_range = [0.1,0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

data_exp = {'pressure': 549300}

fixed_model_set = []

for i in range(len(T_range)):
    for j in range(len(x_range)):
        data_exp['temperature'] = T_range[i]
        data_exp['x_R32'] = x_range[j]
        data_exp['x_emimTf2N'] = 1-x_range[j]
    
        print(data_exp)
        
        model = model_creation.create_model(data_exp)
        
        fixed_model_set.append(model)

{'pressure': 549300, 'temperature': 273, 'x_R32': 0.1, 'x_emimTf2N': 0.9}
2022-10-18 19:29:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 273, 'x_R32': 0.2, 'x_emimTf2N': 0.8}
2022-10-18 19:29:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 273, 'x_R32': 0.3, 'x_emimTf2N': 0.7}
2022-10-18 19:29:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 273, 'x_R32': 0.4, 'x_emimTf2N': 0.6}
2022-10-18 19:30:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 273, 'x_R32': 0.5, 'x_emimTf2N': 0.5}
2022-10-18 19:30:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'tem

2022-10-18 19:30:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 313, 'x_R32': 0.7, 'x_emimTf2N': 0.30000000000000004}
2022-10-18 19:30:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 313, 'x_R32': 0.8, 'x_emimTf2N': 0.19999999999999996}
2022-10-18 19:30:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 313, 'x_R32': 0.9, 'x_emimTf2N': 0.09999999999999998}
2022-10-18 19:30:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 323, 'x_R32': 0.1, 'x_emimTf2N': 0.9}
2022-10-18 19:30:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 323, 'x_R32': 0

2022-10-18 19:30:45 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 360, 'x_R32': 0.3, 'x_emimTf2N': 0.7}
2022-10-18 19:30:46 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 360, 'x_R32': 0.4, 'x_emimTf2N': 0.6}
2022-10-18 19:30:46 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 360, 'x_R32': 0.5, 'x_emimTf2N': 0.5}
2022-10-18 19:30:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 360, 'x_R32': 0.6, 'x_emimTf2N': 0.4}
2022-10-18 19:30:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 360, 'x_R32': 0.7, 'x_emimTf2N': 0.30000000000000004}
2022-10-1

In [16]:
design_range_values = [T_range, x_range]

In [17]:
mbdoe_obj = MBDOE(data_file, model_creation)

all_fim = mbdoe_obj.run_grid_search(design_range_values, fixed_models=fixed_model_set, 
                                    prior_FIM = prior_info, 
                                    scale_opt=True)
                                    #, record_name="SRK_polyTdep", 
                                    # init_temp_option=343, 
                                    # init_pressure_option= 150000,
                                    # init_x_c1_option = 0.4,
                                    # poly_option=True)



parameter set: {'fs.properties.PR_kappa_A["R32", "emimTf2N"]': 0.3212980565927437, 'fs.properties.PR_kappa_A["emimTf2N", "R32"]': 0.8429816989080334, 'fs.properties.PR_kappa_B["R32", "emimTf2N"]': -0.5532108602621684, 'fs.properties.PR_kappa_B["emimTf2N", "R32"]': -0.6100855368792957, 'fs.properties.PR_kappa_C["R32", "emimTf2N"]': 0.1862684932461356, 'fs.properties.PR_kappa_C["emimTf2N", "R32"]': -0.3935451934926509}
All measurements are flattened.
Flatten measurement name: ['fs.state_block.pressure']
90  design vectors will be searched.
=======This is the  1 th iteration=======
Design variable values of this iteration: {'fs.F101.inlet.temperature': {0: 1}, 'fs.F101.inlet.pressure': {0: 1}, 'fs.F101.inlet.mole_frac_comp[0,"R32"]': {0: 1}, "fs.F101.inlet.mole_frac_comp[0,'emimTf2N']": {0: 1}}
Sensitivity information is scaled by its corresponding parameter nominal value.
After practice: {'fs.state_block.pressure': [0]}
name: fs.state_block.pressure
Build time with direct kaug mode [s]: 

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 7.62939453125e-06
Solve time with direct kaug mode [s]: 0.5008559226989746
Total wall clock time [s]: 0.5014605522155762
Existed information has been added.
FIM: [[ 4.63095095e+13 -1.09977334e+13  8.46028351e+13 -8.12305431e+12
   3.03923975e+13 -5.37319942e+12]
 [-1.09977334e+13  3.94414365e+12 -1.93295960e+13  2.82720964e+12
  -6.67542338e+12  1.81146613e+12]
 [ 8.46028351e+13 -1.93295960e+13  1.55417120e+14 -1.43452192e+13
   5.61364254e+13 -9.53613766e+12]
 [-8.12305431e+12  2.82720964e+12 -1.43452192e+13  2.03235198e+12
  -4.97868996e+12  1.30616761e+12]
 [ 3.03923975e+13 -6.67542338e+12  5.61364254e+13 -4.97868996e+12
   2.03848448e+13 -3.32664307e+12]
 [-5.37319942e+12  1.81146613e+12 -9.53613766e+12  1.30616761e+12
  -3.32664307e+12  8.42216341e+11]]
Trace: 228930185958057.0
Determinant: 1.7231013305604597e+61
Condition number: 260804823.602141
Minimal eigen value: 867197.3108821736
Eigen values: [2.26169242e+1

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 7.152557373046875e-06
Solve time with direct kaug mode [s]: 0.515080451965332
Total wall clock time [s]: 0.5157217979431152
Existed information has been added.
FIM: [[ 4.58893995e+13 -1.12354961e+13  8.39404665e+13 -8.28060852e+12
   3.01881755e+13 -5.46625611e+12]
 [-1.12354961e+13  4.50335272e+12 -1.97045074e+13  3.19775595e+12
  -6.79102920e+12  2.03031387e+12]
 [ 8.39404665e+13 -1.97045074e+13  1.54372793e+14 -1.45936555e+13
   5.58144378e+13 -9.68287227e+12]
 [-8.28060852e+12  3.19775595e+12 -1.45936555e+13  2.27788550e+12
  -5.05529653e+12  1.45118175e+12]
 [ 3.01881755e+13 -6.79102920e+12  5.58144378e+13 -5.05529653e+12
   2.02855694e+13 -3.37188942e+12]
 [-5.46625611e+12  2.03031387e+12 -9.68287227e+12  1.45118175e+12
  -3.37188942e+12  9.27862908e+11]]
Trace: 228256863001733.3
Determinant: 4.82501178588089e+61
Condition number: 155230821.55467254
Minimal eigen value: 1448342.9646132595
Eigen values: [2.2482746

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 1.239776611328125e-05
Solve time with direct kaug mode [s]: 0.7149362564086914
Total wall clock time [s]: 0.7157235145568848
Existed information has been added.
FIM: [[ 4.64074145e+13 -1.09548073e+13  8.48052626e+13 -8.09845596e+12
   3.04701734e+13 -5.36102541e+12]
 [-1.09548073e+13  3.91905208e+12 -1.92710761e+13  2.81131711e+12
  -6.66030833e+12  1.80252902e+12]
 [ 8.48052626e+13 -1.92710761e+13  1.55814841e+14 -1.43127275e+13
   5.62841627e+13 -9.52079501e+12]
 [-8.09845596e+12  2.81131711e+12 -1.43127275e+13  2.02232511e+12
  -4.97068306e+12  1.30055411e+12]
 [ 3.04701734e+13 -6.66030833e+12  5.62841627e+13 -4.97068306e+12
   2.04384217e+13 -3.32314371e+12]
 [-5.36102541e+12  1.80252902e+12 -9.52079501e+12  1.30055411e+12
  -3.32314371e+12  8.39089774e+11]]
Trace: 229441144406966.4
Determinant: 1.4250379075638854e+61
Condition number: 275012605.50040275
Minimal eigen value: 824285.5245761025
Eigen values: [2.26688

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 6.67572021484375e-06
Solve time with direct kaug mode [s]: 0.5704474449157715
Total wall clock time [s]: 0.5713005065917969
Existed information has been added.
FIM: [[ 4.70393934e+13 -1.20680109e+13  8.58381911e+13 -8.86314362e+12
   3.08003179e+13 -5.82921834e+12]
 [-1.20680109e+13  4.99365905e+12 -2.10905527e+13  3.54948027e+12
  -7.24185479e+12  2.25447485e+12]
 [ 8.58381911e+13 -2.10905527e+13  1.57503096e+14 -1.55625719e+13
   5.68237624e+13 -1.02860333e+13]
 [-8.86314362e+12  3.54948027e+12 -1.55625719e+13  2.52938012e+12
  -5.37016199e+12  1.61100227e+12]
 [ 3.08003179e+13 -7.24185479e+12  5.68237624e+13 -5.37016199e+12
   2.06108885e+13 -3.56773143e+12]
 [-5.82921834e+12  2.25447485e+12 -1.02860333e+13  1.61100227e+12
  -3.56773143e+12  1.02916394e+12]]
Trace: 233705580720550.62
Determinant: 2.1599258746707234e+61
Condition number: 267064679.1981026
Minimal eigen value: 861103.583456671
Eigen values: [2.2997035

...

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 8.106231689453125e-06
Solve time with direct kaug mode [s]: 0.5738780498504639
Total wall clock time [s]: 0.5746283531188965
Existed information has been added.
FIM: [[ 4.60305031e+13 -1.08125786e+13  8.43732640e+13 -8.00354020e+12
   3.04054768e+13 -5.30503318e+12]
 [-1.08125786e+13  3.88940369e+12 -1.90452365e+13  2.79105379e+12
  -6.59076686e+12  1.79020047e+12]
 [ 8.43732640e+13 -1.90452365e+13  1.55484003e+14 -1.41632633e+13
   5.63280311e+13 -9.43360433e+12]
 [-8.00354020e+12  2.79105379e+12 -1.41632633e+13  2.00849324e+12
  -4.92517203e+12  1.29215174e+12]
 [ 3.04054768e+13 -6.59076686e+12  5.63280311e+13 -4.92517203e+12
   2.05120943e+13 -3.29700039e+12]
 [-5.30503318e+12  1.79020047e+12 -9.43360433e+12  1.29215174e+12
  -3.29700039e+12  8.33995742e+11]]
Trace: 228758493215042.0
Determinant: 1.4478223678110146e+61
Condition number: 240797728.73026347
Minimal eigen value: 938449.1769126306
Eigen values: [2.25976

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 9.059906005859375e-06
Solve time with direct kaug mode [s]: 0.6701576709747314
Total wall clock time [s]: 0.6710729598999023
Existed information has been added.
FIM: [[ 7.01085175e+13 -1.81456114e+13  1.33460327e+14 -1.42866363e+13
   4.99749682e+13 -1.01034180e+13]
 [-1.81456114e+13  6.09048794e+12 -3.39948724e+13  4.67698437e+12
  -1.25507318e+13  3.23047751e+12]
 [ 1.33460327e+14 -3.39948724e+13  2.55556200e+14 -2.69724244e+13
   9.62237156e+13 -1.92159286e+13]
 [-1.42866363e+13  4.67698437e+12 -2.69724244e+13  3.62439420e+12
  -1.00317948e+13  2.52620832e+12]
 [ 4.99749682e+13 -1.25507318e+13  9.62237156e+13 -1.00317948e+13
   3.64172677e+13 -7.19691544e+12]
 [-1.01034180e+13  3.23047751e+12 -1.92159286e+13  2.52620832e+12
  -7.19691544e+12  1.77643942e+12]]
Trace: 373573306712034.56
Determinant: 1.2939396609880261e+63
Condition number: 212677893.5930679
Minimal eigen value: 1741662.253075491
Eigen values: [3.70413

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 7.152557373046875e-06
Solve time with direct kaug mode [s]: 0.6732454299926758
Total wall clock time [s]: 0.6741354465484619
Existed information has been added.
FIM: [[ 1.00297544e+16 -1.08605868e+16  2.04374697e+16 -9.30475962e+15
   8.14439652e+15 -7.10555231e+15]
 [-1.08605868e+16  1.17943380e+16 -2.21376959e+16  1.01055750e+16
  -8.82443995e+15  7.71762824e+15]
 [ 2.04374697e+16 -2.21376959e+16  4.16478709e+16 -1.89667882e+16
   1.65977352e+16 -1.44841947e+16]
 [-9.30475962e+15  1.01055750e+16 -1.89667882e+16  8.65869725e+15
  -7.56060253e+15  6.61269834e+15]
 [ 8.14439652e+15 -8.82443995e+15  1.65977352e+16 -7.56060253e+15
   6.61493033e+15 -5.77382353e+15]
 [-7.10555231e+15  7.71762824e+15 -1.44841947e+16  6.61269834e+15
  -5.77382353e+15  5.05018866e+15]]
Trace: 8.379577947490462e+16
Determinant: 1.2130715995632623e+67
Condition number: 41146923405.39716
Minimal eigen value: 2035425.709836967
Eigen values: [8.37

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 8.106231689453125e-06
Solve time with direct kaug mode [s]: 0.6571977138519287
Total wall clock time [s]: 0.6580185890197754
Existed information has been added.
FIM: [[ 6.06185564e+13 -1.39114927e+13  1.14721757e+14 -1.07116032e+13
   4.27518039e+13 -7.41433292e+12]
 [-1.39114927e+13  4.53396393e+12 -2.54888792e+13  3.35426801e+12
  -9.21087131e+12  2.22884714e+12]
 [ 1.14721757e+14 -2.54888792e+13  2.18619220e+14 -1.97941884e+13
   8.20122996e+13 -1.38195009e+13]
 [-1.07116032e+13  3.35426801e+12 -1.97941884e+13  2.50062743e+12
  -7.21480568e+12  1.67543918e+12]
 [ 4.27518039e+13 -9.21087131e+12  8.20122996e+13 -7.21480568e+12
   3.09606847e+13 -5.08037981e+12]
 [-7.41433292e+12  2.22884714e+12 -1.38195009e+13  1.67543918e+12
  -5.08037981e+12  1.13251028e+12]]
Trace: 318365562479986.75
Determinant: 4.453637208537529e+62
Condition number: 220989657.89632022
Minimal eigen value: 1426897.912850047
Eigen values: [3.15329

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 8.821487426757812e-06
Solve time with direct kaug mode [s]: 0.6576616764068604
Total wall clock time [s]: 0.6584875583648682
Existed information has been added.
The condition number is: 11981971897346.863 ;
A condition number bigger than  1000000000000.0  is considered near singular.
FIM: [[ 3.72557049e+18 -3.15360854e+18  7.74545906e+18 -2.75579769e+18
   3.14896434e+18 -2.14643736e+18]
 [-3.15360854e+18  2.66947436e+18 -6.55635660e+18  2.33273506e+18
  -2.66552952e+18  1.81692249e+18]
 [ 7.74545906e+18 -6.55635660e+18  1.61028088e+19 -5.72930780e+18
   6.54669787e+18 -4.46244704e+18]
 [-2.75579769e+18  2.33273506e+18 -5.72930780e+18  2.03847364e+18
  -2.32928761e+18  1.58772799e+18]
 [ 3.14896434e+18 -2.66552952e+18  6.54669787e+18 -2.32928761e+18
   2.66160146e+18 -1.81423718e+18]
 [-2.14643736e+18  1.81692249e+18 -4.46244704e+18  1.58772799e+18
  -1.81423718e+18  1.23665090e+18]]
Trace: 2.843457968812187e+19
Determ

In [18]:
fixed = {}

all_fim.extract_criteria()
print(all_fim.store_all_results_dataframe)

all_fim.figure_drawing(fixed, ['Temperature','x_R32'], 'PR quadratic','Temperature [K]', 'x (R32)' )


Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.1)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.2)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.3)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.4)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.5)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.6)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.7)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.8)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.9)
Design variable:  ['Temperature', 'x_R32']
Value          :  (283, 0.1)
Design variable:  ['Temperature', 'x_R32']
Value          :  (283, 0.2)
Design variable:  ['Temperature', 'x_R32']
Value          :  (283, 0.3)
Design variable:  ['Temperature', 'x_R32']
Value          :  (283, 0.4)
Design variable:  ['Temperature', 'x_R32']
Value          :  (28